Set up virtual environment and import necessary modules.

In [ ]:
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp
!pip install streamlit
!pip install googletrans

Import necessary Python modules.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

import pandas as pd
import numpy as np
import json

import googletrans
from googletrans import Translator
translator = Translator()

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
spark = sparknlp.start()

In [ ]:
text_list = ["""Уильям Генри Гейтс III (родился 28 октября 1955 года) - американский бизнес-магнат, разработчик программного обеспечения, инвестор и филантроп. Он наиболее известен как соучредитель корпорации Microsoft. За время своей карьеры в Microsoft Гейтс занимал должности председателя, главного исполнительного директора (CEO), президента и главного архитектора программного обеспечения, а также был крупнейшим индивидуальным акционером до мая 2014 года. Он является одним из самых известных предпринимателей и пионеров микрокомпьютерная революция 1970-х и 1980-х годов. Гейтс родился и вырос в Сиэтле, штат Вашингтон, в 1975 году вместе с другом детства Полом Алленом в Альбукерке, штат Нью-Мексико, и основал компанию Microsoft. она стала крупнейшей в мире компанией-разработчиком программного обеспечения для персональных компьютеров. Гейтс руководил компанией в качестве председателя и генерального директора, пока в январе 2000 года не ушел с поста генерального директора, но остался председателем и стал главным архитектором программного обеспечения. В конце 1990-х Гейтс подвергся критике за свою деловую тактику, которая считалась антиконкурентной. Это мнение было подтверждено многочисленными судебными решениями. В июне 2006 года Гейтс объявил, что перейдет на неполный рабочий день в Microsoft и будет работать на полную ставку в Фонде Билла и Мелинды Гейтс, частном благотворительном фонде, который он и его жена Мелинда Гейтс создали в 2000 году. [ 9] Постепенно он передал свои обязанности Рэю Оззи и Крейгу Манди. Он ушел с поста президента Microsoft в феврале 2014 года и занял новую должность консультанта по технологиям для поддержки вновь назначенного генерального директора Сатья Наделла.""",
             """Мона Лиза - картина маслом 16-го века, созданная Леонардо. Он проводится в Лувре в Париже.""",
]

Change `MODEL_NAME` to use a different model. Change `LANG` to the language code of the language you want to use.

In [ ]:
MODEL_NAME = 'wikiner_6B_100'
# MODEL_NAME = 'wikiner_6B_300'
# MODEL_NAME = 'wikiner_840B_300'

LANG = 'ru'


Create the pipeline to process the inputs. Selects a different `WordEmbeddingsModel` depending on the NER model chosen above.

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Select the appropriate WordEmbeddingsModel to match the NerDLModel.
embeddings = None
if MODEL_NAME == 'wikiner_6B_100':
    embeddings = WordEmbeddingsModel.pretrained(name='glove_100d') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")
elif MODEL_NAME=='wikiner_6B_300':
    embeddings = WordEmbeddingsModel.pretrained(name='glove_6B_300', lang='xx') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")
elif MODEL_NAME=='wikiner_840B_300':
    embeddings = WordEmbeddingsModel.pretrained(name='glove_840B_300', lang='xx') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")

public_ner = NerDLModel.pretrained(MODEL_NAME, lang=LANG) \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               tokenizer,
                               embeddings,
                               public_ner,
                               ner_converter])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

Use the pipeline to create outputs for the demo.

In [ ]:
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipelineModel.transform(df)
result.select(F.explode(
    F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')) \
        .alias('cols')).select(F.expr("cols['0']").alias('chunk'),
    F.expr("cols['1']['entity']").alias('ner_label')).show(truncate=False)
result = result.toPandas()